In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

from tensorflow.keras.layers import Input, Dense, ReLU, LeakyReLU, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD, Adam

In [2]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train / 255.0 * 2 - 1
x_test = x_test / 255.0 * 2 - 1

In [3]:
x_train.shape

(60000, 28, 28)

In [4]:
n, H, W = x_train.shape

In [5]:
D = H*W

In [6]:
x_train = x_train.reshape(-1, D)
x_test  = x_test.reshape(-1, D)

In [7]:
latent_dim = 100

In [8]:
i = Input(shape=(latent_dim,))
x = Dense(256, activation=LeakyReLU(alpha=0.2))(i)
x = BatchNormalization(momentum=0.7)(x)
x = Dense(512, activation=LeakyReLU(alpha=0.2))(x)
x = BatchNormalization(momentum=0.7)(x)
x = Dense(1024, activation=LeakyReLU(alpha=0.2))(x)
x = BatchNormalization(momentum=0.7)(x)
x = Dense(D, activation='tanh')(x)

generator = Model(i, x)

2022-01-14 20:16:34.770871: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-01-14 20:16:34.770924: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: jupyter-alan-20chen
2022-01-14 20:16:34.770938: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: jupyter-alan-20chen
2022-01-14 20:16:34.771054: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 460.91.3
2022-01-14 20:16:34.771082: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 460.91.3
2022-01-14 20:16:34.771094: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 460.91.3
2022-01-14 20:16:34.771594: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the followin

In [9]:
i = Input(shape=(D,))
x = Dense(512, activation=LeakyReLU(alpha=0.2))(i)
x = Dense(256, activation=LeakyReLU(alpha=0.2))(x)
x = Dense(1, activation='sigmoid')(x)

discriminator = Model(i, x)

In [10]:
discriminator.compile(
    loss='binary_crossentropy',
    optimizer=Adam(0.0002, 0.5),
    metrics=['accuracy'])

In [11]:
z = Input(shape=(latent_dim,))
img = generator(z)
discriminator.trainable = False
fake_pred = discriminator(img)
combined_model = Model(z, fake_pred)

In [12]:
combined_model.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5))

In [13]:
batch_size = 32
epochs = 30000
sample_period = 200

ones = np.ones(batch_size)
zeros = np.zeros(batch_size)

if not os.path.exists('gan_images'):
    os.makedirs('gan_images')

In [14]:
def sample_images(epoch):
    rows, cols = 5, 5
    noise = np.random.randn(rows * cols, latent_dim)
    imgs = generator.predict(noise)

    # Rescale images 0 - 1
    imgs = 0.5 * imgs + 0.5

    fig, axs = plt.subplots(rows, cols)
    idx = 0
    for i in range(rows):
        for j in range(cols):
            axs[i, j].imshow(imgs[idx].reshape(H, W), cmap='gray')
            axs[i, j].axis('off')
            idx += 1
    fig.savefig("gan_images/%d.png" % epoch)
    plt.close()

In [15]:
for epoch in range(epochs):

    idx = np.random.randint(0, x_train.shape[0], batch_size)
    real_imgs = x_train[idx]

    noise = np.random.randn(batch_size, latent_dim)
    fake_imgs = generator.predict(noise)

    # train discriminator
    discriminator.train_on_batch(real_imgs, ones)
    discriminator.train_on_batch(fake_imgs, zeros)

    # train generator
    noise = np.random.randn(batch_size, latent_dim)
    combined_model.train_on_batch(noise, ones)

    noise = np.random.randn(batch_size, latent_dim)
    combined_model.train_on_batch(noise, ones)

    if epoch % sample_period == 0:
        sample_images(epoch)
        print("epoch: "+str(epoch))

2022-01-14 20:16:34.988791: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


epoch: 0
epoch: 200
epoch: 400
epoch: 600
epoch: 800
epoch: 1000
epoch: 1200
epoch: 1400
epoch: 1600
epoch: 1800
epoch: 2000
epoch: 2200
epoch: 2400
epoch: 2600
epoch: 2800
epoch: 3000
epoch: 3200
epoch: 3400
epoch: 3600
epoch: 3800
epoch: 4000
epoch: 4200
epoch: 4400
epoch: 4600
epoch: 4800
epoch: 5000
epoch: 5200
epoch: 5400
epoch: 5600
epoch: 5800
epoch: 6000
epoch: 6200
epoch: 6400
epoch: 6600
epoch: 6800
epoch: 7000
epoch: 7200
epoch: 7400
epoch: 7600
epoch: 7800
epoch: 8000
epoch: 8200
epoch: 8400
epoch: 8600
epoch: 8800
epoch: 9000
epoch: 9200
epoch: 9400
epoch: 9600
epoch: 9800
epoch: 10000
epoch: 10200
epoch: 10400
epoch: 10600
epoch: 10800
epoch: 11000
epoch: 11200
epoch: 11400
epoch: 11600
epoch: 11800
epoch: 12000


2022-01-14 20:37:49.845655: W tensorflow/core/data/root_dataset.cc:167] Optimization loop failed: Cancelled: Operation was cancelled


epoch: 12200
epoch: 12400
epoch: 12600
epoch: 12800
epoch: 13000
epoch: 13200
epoch: 13400
epoch: 13600
epoch: 13800
epoch: 14000
epoch: 14200
epoch: 14400
epoch: 14600
epoch: 14800
epoch: 15000
epoch: 15200
epoch: 15400
epoch: 15600
epoch: 15800
epoch: 16000
epoch: 16200
epoch: 16400
epoch: 16600
epoch: 16800
epoch: 17000
epoch: 17200
epoch: 17400
epoch: 17600
epoch: 17800
epoch: 18000
epoch: 18200
epoch: 18400
epoch: 18600
epoch: 18800
epoch: 19000
epoch: 19200
epoch: 19400
epoch: 19600
epoch: 19800
epoch: 20000
epoch: 20200
epoch: 20400
epoch: 20600
epoch: 20800
epoch: 21000
epoch: 21200
epoch: 21400
epoch: 21600
epoch: 21800
epoch: 22000
epoch: 22200
epoch: 22400
epoch: 22600
epoch: 22800
epoch: 23000
epoch: 23200
epoch: 23400
epoch: 23600
epoch: 23800
epoch: 24000
epoch: 24200
epoch: 24400
epoch: 24600
epoch: 24800
epoch: 25000
epoch: 25200
epoch: 25400
epoch: 25600
epoch: 25800
epoch: 26000
epoch: 26200
epoch: 26400
epoch: 26600
epoch: 26800
epoch: 27000
epoch: 27200
epoch: 27400

In [16]:
combined_model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 100)]             0         
_________________________________________________________________
model (Functional)           (None, 784)               1493520   
_________________________________________________________________
model_1 (Functional)         (None, 1)                 533505    
Total params: 2,027,025
Trainable params: 1,489,936
Non-trainable params: 537,089
_________________________________________________________________
